# Part 4: Pandas DataFrames - Data Loading and Manipulation

In this notebook, we'll dive deep into Pandas DataFrames, learning how to load data and perform essential data manipulation operations.

## Topics Covered:
- Loading data from CSV, TSV, and TXT files
- Selecting columns and rows
- Filtering data
- Dropping missing values and columns
- Grouping data
- Joining/Merging DataFrames

In [ ]:
import pandas as pd
import numpy as np

# For this workshop, we'll create sample data
# In real scenarios, you'd load actual files
print("Pandas version:", pd.__version__)

## 1. Loading Data from Files

Pandas can read data from various file formats:

In [1]:
# Loading CSV files (Comma-Separated Values)
demo_df = pd.read_csv('../data/demo_data.csv')
print("Demographic data loaded from CSV:")
print(demo_df.head())
print(f"\nShape: {demo_df.shape}")

# Loading TSV files (Tab-Separated Values)
demo_tsv = pd.read_csv('../data/demo_data.tsv', sep='\t')
print("\n\nSame data loaded from TSV:")
print(demo_tsv.head(3))

# Loading TXT files with custom delimiter
demo_txt = pd.read_csv('../data/demo_data.txt', sep=' ')
print("\n\nSame data loaded from TXT (space-delimited):")
print(demo_txt.head(3))

# Common parameters:
# - header: row number to use as column names (default: 0)
# - index_col: column to use as row labels
# - usecols: list of columns to read
# - na_values: values to recognize as NA/NaN

print("\n\nAll three methods load the same data!")
print(f"CSV shape: {demo_df.shape}, TSV shape: {demo_tsv.shape}, TXT shape: {demo_txt.shape}")

NameError: name 'pd' is not defined

## 2. Loading Multiple Related Datasets

In research, data is often split across multiple files. Let's load all our datasets:

In [ ]:
# Load all datasets
demo_df = pd.read_csv('../data/demo_data.csv')
neuropsych_df = pd.read_csv('../data/neuropsych_data.csv')
imaging_df = pd.read_csv('../data/imaging_data.csv')
biomarker_df = pd.read_csv('../data/biomarker_data.csv')
clinical_df = pd.read_csv('../data/clinical_data.csv')

print("Demographic data:")
print(demo_df.head())
print(f"Shape: {demo_df.shape}\n")

print("Neuropsychological data:")
print(neuropsych_df.head())
print(f"Shape: {neuropsych_df.shape}\n")

print("Imaging data (note: only half of participants have imaging):")
print(imaging_df.head())
print(f"Shape: {imaging_df.shape}\n")

print("Biomarker data:")
print(biomarker_df.head())
print(f"Shape: {biomarker_df.shape}\n")

print("Clinical data:")
print(clinical_df.head())
print(f"Shape: {clinical_df.shape}")

## 3. Selecting Columns and Rows

There are multiple ways to select data from a DataFrame:

In [ ]:
# Select a single column (returns a Series)
ages = demo_df['age']
print("Age column (Series):")
print(ages.head())
print(f"\nType: {type(ages)}")

# Select multiple columns (returns a DataFrame)
subset = demo_df[['record_id', 'age', 'sex']]
print("\n\nMultiple columns:")
print(subset.head())
print(f"Type: {type(subset)}")

In [ ]:
# Select rows by index position with iloc
print("First row:")
print(demo_df.iloc[0])

print("\n\nFirst 5 rows:")
print(demo_df.iloc[0:5])

print("\n\nSpecific rows and columns:")
print(demo_df.iloc[0:3, 1:4])  # Rows 0-2, Columns 1-3 (age, sex, education_level)

In [ ]:
# Select rows by label with loc
print("Using loc with row indices and column names:")
print(demo_df.loc[0:4, ['record_id', 'age', 'education_level']])

# Boolean indexing - more powerful for filtering
print("\n\nRows where age > 70:")
elderly = demo_df.loc[demo_df['age'] > 70]
print(elderly.head())
print(f"Number of elderly participants: {len(elderly)}")

## 4. Filtering Data

Filtering allows you to extract rows based on conditions:

In [ ]:
# Single condition - filter by education level
phd_participants = demo_df[demo_df['education_level'] == 'phd']
print(f"PhD participants: {len(phd_participants)} records")
print(phd_participants.head())

# Multiple conditions with & (and) and | (or)
# Note: Use & instead of 'and', | instead of 'or'
# Wrap each condition in parentheses

# Young females with bachelor's degree or higher
young_educated_females = demo_df[
    (demo_df['age'] < 30) & 
    (demo_df['sex'] == 'F') & 
    (demo_df['education_level'].isin(['bachelor', 'master', 'phd']))
]
print(f"\n\nYoung educated females: {len(young_educated_females)} records")
print(young_educated_females.head())

In [ ]:
# Using isin() for multiple values
bachelor_or_master = demo_df[demo_df['education_level'].isin(['bachelor', 'master'])]
print(f"Participants with bachelor or master: {len(bachelor_or_master)} records")
print(f"Breakdown by education:")
print(bachelor_or_master['education_level'].value_counts())

# Filter the clinical data by health conditions
print("\n\nClinical data filtering:")
# Participants with diabetes
diabetes_patients = clinical_df[clinical_df['diabetes'] == 1]
print(f"Participants with diabetes: {len(diabetes_patients)}")

# Participants with high cholesterol (>240 mg/dL)
high_cholesterol = clinical_df[clinical_df['cholesterol_mg_dl'] > 240]
print(f"Participants with high cholesterol: {len(high_cholesterol)}")

# Participants with MCI (Mild Cognitive Impairment)
mci_patients = clinical_df[clinical_df['mild_cognitive_impairment'] == 1]
print(f"Participants with MCI: {len(mci_patients)}")

## 5. Handling Missing Values

Real-world data often has missing values that need to be handled. 
Note: Our imaging and biomarker data only have measurements for half the participants!

In [ ]:
# First, let's merge some datasets to create missing values
# Merge demographic data with imaging data (left join - keeps all demographics)
demo_with_imaging = demo_df.merge(imaging_df, on='record_id', how='left')

print("After merging demographics with imaging:")
print(demo_with_imaging.head(10))

# Check for missing values
print("\n\nMissing values per column:")
print(demo_with_imaging.isnull().sum())

print("\n\nPercentage of missing values:")
print((demo_with_imaging.isnull().sum() / len(demo_with_imaging)) * 100)

# Check which rows have any missing values
rows_with_na = demo_with_imaging[demo_with_imaging.isnull().any(axis=1)]
print(f"\n\nRows with missing values: {len(rows_with_na)}")
print(rows_with_na.head())

In [ ]:
# Drop rows with any missing values
df_no_na = demo_with_imaging.dropna()
print(f"Original shape: {demo_with_imaging.shape}")
print(f"After dropping NA: {df_no_na.shape}")
print(f"Lost {demo_with_imaging.shape[0] - df_no_na.shape[0]} rows")

# Drop rows where specific columns have missing values
# Only drop if imaging data is missing
df_imaging_only = demo_with_imaging.dropna(subset=['hippocampus_volume', 'amygdala_volume'])
print(f"\nAfter dropping rows with missing imaging: {df_imaging_only.shape}")

# Fill missing values with mean (common strategy for numerical data)
df_filled = demo_with_imaging.copy()
print(f"\n\nBefore filling - missing hippocampus volumes: {df_filled['hippocampus_volume'].isnull().sum()}")

df_filled['hippocampus_volume'].fillna(df_filled['hippocampus_volume'].mean(), inplace=True)
df_filled['amygdala_volume'].fillna(df_filled['amygdala_volume'].mean(), inplace=True)
df_filled['cortical_thickness'].fillna(df_filled['cortical_thickness'].mean(), inplace=True)

print(f"After filling - missing hippocampus volumes: {df_filled['hippocampus_volume'].isnull().sum()}")

# Alternative: Fill with median or a specific value
# df['column'].fillna(df['column'].median(), inplace=True)
# df['column'].fillna(0, inplace=True)

## 6. Dropping Columns

Sometimes you need to remove columns from your DataFrame:

In [ ]:
# Drop a single column
clinical_subset = clinical_df.drop('smoking_status', axis=1)
# axis=1 means drop column, axis=0 would drop rows
print("Original columns:")
print(clinical_df.columns.tolist())
print("\nColumns after dropping smoking_status:")
print(clinical_subset.columns.tolist())

# Drop multiple columns
neuropsych_minimal = neuropsych_df.drop(['attention_score', 'executive_function_score'], axis=1)
print("\n\nNeuropsych columns after dropping:")
print(neuropsych_minimal.columns.tolist())
print(neuropsych_minimal.head())

# Note: Original df is unchanged unless you use inplace=True
print(f"\n\nOriginal neuropsych_df still has {neuropsych_df.shape[1]} columns")
# To modify the original: neuropsych_df.drop('column', axis=1, inplace=True)

## 7. Grouping Data

Group by allows you to aggregate data based on categories:

In [ ]:
# Group by a single column - average age by education level
age_by_education = demo_df.groupby('education_level')['age'].mean()
print("Average age by education level:")
print(age_by_education)

# Multiple aggregations
education_stats = demo_df.groupby('education_level')['age'].agg(['mean', 'median', 'std', 'count'])
print("\n\nAge statistics by education level:")
print(education_stats.round(2))

In [ ]:
# Group by multiple columns - count participants by sex and education
sex_edu_counts = demo_df.groupby(['sex', 'education_level']).size()
print("Participant counts by sex and education:")
print(sex_edu_counts)

# Unstack for better readability (create a pivot table)
print("\n\nUnstacked view (crosstab format):")
print(sex_edu_counts.unstack())

# Alternative: use value_counts with multiple columns
print("\n\nUsing value_counts:")
print(demo_df.value_counts(['sex', 'education_level']).sort_index())

In [ ]:
# Custom aggregation functions on clinical data
# Group by diabetes status and get various health metrics
clinical_by_diabetes = clinical_df.groupby('diabetes').agg({
    'cholesterol_mg_dl': ['mean', 'std'],
    'systolic_bp_mm_hg': ['mean', 'std'],
    'bmi': ['mean', 'std'],
    'mild_cognitive_impairment': 'sum'  # Count how many have MCI in each group
})

print("Health metrics by diabetes status:")
print(clinical_by_diabetes.round(2))

# Group by smoking status
print("\n\nHealth metrics by smoking status:")
smoking_stats = clinical_df.groupby('smoking_status').agg({
    'cholesterol_mg_dl': 'mean',
    'systolic_bp_mm_hg': 'mean',
    'bmi': 'mean',
    'diabetes': 'sum'
}).round(2)
print(smoking_stats)

## 8. Joining DataFrames

Combining data from multiple DataFrames is essential for research data analysis.
Let's explore all four types of joins with our datasets:

In [ ]:
# Let's understand our data before joining
print("Dataset sizes:")
print(f"Demographics: {demo_df.shape[0]} participants")
print(f"Neuropsych: {neuropsych_df.shape[0]} participants")
print(f"Imaging: {imaging_df.shape[0]} participants")
print(f"Biomarker: {biomarker_df.shape[0]} participants")
print(f"Clinical: {clinical_df.shape[0]} participants")

print("\n\nFirst few record_ids from each dataset:")
print("Demo:", demo_df['record_id'].head(3).tolist())
print("Imaging:", imaging_df['record_id'].head(3).tolist())
print("Biomarker:", biomarker_df['record_id'].head(3).tolist())

In [ ]:
# INNER JOIN - only participants who have BOTH demographic AND imaging data
# This keeps only the 500 participants who have imaging scans
inner_join = pd.merge(demo_df, imaging_df, on='record_id', how='inner')
print("Inner Join (participants with both demo AND imaging):")
print(f"Shape: {inner_join.shape}")
print(inner_join.head())
print(f"\n{len(inner_join)} participants have complete data in both datasets")

In [ ]:
# LEFT JOIN - keep ALL demographic data, add imaging where available
# This is most common in research - keep all participants, add measurements where available
left_join = pd.merge(demo_df, imaging_df, on='record_id', how='left')
print("Left Join (all demographics, imaging where available):")
print(f"Shape: {left_join.shape}")
print(left_join.head(10))

# Check for missing values (participants without imaging)
print(f"\nParticipants without imaging data: {left_join['hippocampus_volume'].isnull().sum()}")
print(f"Participants with imaging data: {left_join['hippocampus_volume'].notnull().sum()}")

In [ ]:
# RIGHT JOIN - keep ALL imaging data, add demographics where available
# In this case, same as inner join since all imaging participants have demographics
right_join = pd.merge(demo_df, imaging_df, on='record_id', how='right')
print("Right Join (all imaging, demographics where available):")
print(f"Shape: {right_join.shape}")
print(right_join.head())

# Since all imaging participants also have demographics, no missing values
print(f"\nMissing demographic data: {right_join['age'].isnull().sum()}")

In [ ]:
# OUTER JOIN - keep ALL rows from both DataFrames
# In this case, same as left join since imaging is a subset of demographics
outer_join = pd.merge(demo_df, imaging_df, on='record_id', how='outer')
print("Outer Join (all rows from both datasets):")
print(f"Shape: {outer_join.shape}")
print(outer_join.head(10))

print(f"\nTotal participants: {len(outer_join)}")
print(f"With imaging: {outer_join['hippocampus_volume'].notnull().sum()}")
print(f"Without imaging: {outer_join['hippocampus_volume'].isnull().sum()}")

### Joining Multiple DataFrames

In research, we often need to combine many datasets. Let's merge everything together:

In [ ]:
# Merge all datasets together - chain multiple merges
# Start with demographics, then add each dataset using left joins
complete_data = demo_df \
    .merge(neuropsych_df, on='record_id', how='left') \
    .merge(imaging_df, on='record_id', how='left') \
    .merge(biomarker_df, on='record_id', how='left') \
    .merge(clinical_df, on='record_id', how='left')

print("Complete merged dataset:")
print(f"Shape: {complete_data.shape}")
print(f"Columns: {complete_data.columns.tolist()}")

print("\n\nFirst few rows:")
print(complete_data.head())

print("\n\nMissing value summary:")
print(complete_data.isnull().sum())

## 9. Practical Example: Complete Data Analysis Pipeline

In [ ]:
# Real-world research scenario: Analyzing cognitive health

# 1. Load all data
demo = pd.read_csv('../data/demo_data.csv')
neuropsych = pd.read_csv('../data/neuropsych_data.csv')
imaging = pd.read_csv('../data/imaging_data.csv')
clinical = pd.read_csv('../data/clinical_data.csv')

print("Step 1: Data loaded")
print(f"Demographics: {demo.shape}, Neuropsych: {neuropsych.shape}")
print(f"Imaging: {imaging.shape}, Clinical: {clinical.shape}")

# 2. Merge datasets (left join to keep all participants)
analysis_df = demo \
    .merge(neuropsych, on='record_id', how='left') \
    .merge(imaging, on='record_id', how='left') \
    .merge(clinical, on='record_id', how='left')

print(f"\nStep 2: Merged data shape: {analysis_df.shape}")

# 3. Filter for complete cases (participants with all measurements)
complete_cases = analysis_df.dropna()
print(f"\nStep 3: Complete cases: {complete_cases.shape[0]} participants")

# 4. Filter for specific research criteria
# Example: Adults 40-70, with complete imaging and neuropsych data
study_sample = complete_cases[
    (complete_cases['age'] >= 40) & 
    (complete_cases['age'] <= 70)
]
print(f"\nStep 4: Study sample (age 40-70): {study_sample.shape[0]} participants")

# 5. Group and analyze by key variables
print("\n\nStep 5: Analysis Results")
print("="*60)

# Cognitive scores by MCI status
print("\nMemory scores by MCI status:")
mci_memory = study_sample.groupby('mild_cognitive_impairment')['memory_score'].agg(['mean', 'std', 'count'])
print(mci_memory.round(2))

# Brain volumes by sex
print("\n\nHippocampus volume by sex:")
hip_by_sex = study_sample.groupby('sex')['hippocampus_volume'].agg(['mean', 'std', 'count'])
print(hip_by_sex.round(2))

# Health metrics by education level
print("\n\nCholesterol by education level:")
chol_by_edu = study_sample.groupby('education_level')['cholesterol_mg_dl'].agg(['mean', 'std', 'count'])
print(chol_by_edu.round(2))

# 6. Create summary statistics for paper
print("\n\nStep 6: Sample Characteristics Table")
print("="*60)
summary_stats = study_sample[['age', 'memory_score', 'hippocampus_volume', 'cholesterol_mg_dl', 'bmi']].describe()
print(summary_stats.round(2))

## 10. Practice Exercises

Now it's your turn! Use the datasets we've loaded to practice your skills:

In [ ]:
# Exercise 1: From the clinical_df, filter for participants who:
# - Have diabetes (diabetes == 1)
# - Are current smokers
# - Have BMI > 30
# Print the shape and first 10 rows

# Your code here:

In [ ]:
# Exercise 2: Using the neuropsych_df, create a summary showing:
# - Mean of all three cognitive scores
# - Standard deviation of all three scores
# - Minimum and maximum values
# Hint: Use .agg() with a list of functions

# Your code here:

In [ ]:
# Exercise 3: Merge demo_df with clinical_df
# Then group by education_level and calculate:
# - Average cholesterol
# - Average BMI
# - Count of participants with diabetes (sum the diabetes column)
# Print the results sorted by education level

# Your code here:

In [ ]:
# Exercise 4 (Bonus): Create a complete dataset by merging all 5 dataframes
# Then filter for participants who:
# - Are female
# - Have master's or PhD education
# - Have imaging data (hippocampus_volume is not null)
# - Have memory_score > 100
# Calculate the average hippocampus_volume for this group

# Your code here: